# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortaleza,-3.7227,-38.5247,27.34,83,40,5.14,BR,1699915816
1,1,ixtapa,20.7000,-105.2000,30.68,72,18,2.68,MX,1699916014
2,2,cuamba,-14.8031,36.5372,21.98,85,70,0.96,MZ,1699916061
3,3,pelabuhanratu,-6.9875,106.5514,25.62,91,76,1.51,ID,1699916061
4,4,iskateley,67.6803,53.1512,-13.32,97,100,3.58,RU,1699915550


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points('Lng','Lat', geo=True, tiles='OSM', size='Humidity',
                                     scale=1, color='City')
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
warm_df=city_data_df.loc[(city_data_df['Max Temp']>= 20) & (city_data_df['Max Temp'] <= 30)]
# Drop any rows with null values
warm_df=warm_df.dropna()
# Display sample data
warm_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortaleza,-3.7227,-38.5247,27.34,83,40,5.14,BR,1699915816
2,2,cuamba,-14.8031,36.5372,21.98,85,70,0.96,MZ,1699916061
3,3,pelabuhanratu,-6.9875,106.5514,25.62,91,76,1.51,ID,1699916061
10,10,farafangana,-22.8167,47.8333,22.25,91,28,2.55,MG,1699915589
13,13,adamstown,-25.0660,-130.1015,22.86,86,95,8.17,PN,1699915955


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=warm_df[['City','Lat','Lng','Humidity','Country']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
0,fortaleza,-3.7227,-38.5247,83,BR,
2,cuamba,-14.8031,36.5372,85,MZ,
3,pelabuhanratu,-6.9875,106.5514,91,ID,
10,farafangana,-22.8167,47.8333,91,MG,
13,adamstown,-25.0660,-130.1015,86,PN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius =10000
params = {
    "apiKey":geoapify_key,
    'categories':'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
fortaleza - nearest hotel: Hotel Catedral
cuamba - nearest hotel: Pensao Cariaco
pelabuhanratu - nearest hotel: Hotel Augusta
farafangana - nearest hotel: Le Concombre Masqué
adamstown - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
avarua - nearest hotel: Paradise Inn
santo antonio - nearest hotel: Pousada Onça Puma
lihue - nearest hotel: Kauai Palms
mawlamyinegyunn - nearest hotel: No hotel found
treasure island - nearest hotel: Thunderbird Beach Resort
alofi - nearest hotel: Matavai Resort
hawaiian paradise park - nearest hotel: No hotel found
merizo village - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
dumbea - nearest hotel: Hôtel Quality Resort Rivland
simpang - nearest hotel: No hotel found
tamandare - nearest hotel: Pousada Baia dos Corais
el nido - nearest hotel: Dragonfly
uturoa - nearest hotel: Hawaiki Nui hotel
saipan - nearest hotel: Chalan Kanoa Beach Hotel
betio village - nearest hotel: Sea

,City,Lat,Lng,Humidity,Country,Hotel Name
0,fortaleza,-3.7227,-38.5247,83,BR,Hotel Catedral
2,cuamba,-14.8031,36.5372,85,MZ,Pensao Cariaco
3,pelabuhanratu,-6.9875,106.5514,91,ID,Hotel Augusta
10,farafangana,-22.8167,47.8333,91,MG,Le Concombre Masqué
13,adamstown,-25.0660,-130.1015,86,PN,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotels_map=hotel_df.hvplot.points('Lng','Lat', geo=True, tiles='OSM', size='Humidity',
                                     color='City', hover_cols=['Hotel Name', 'Country'])
# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)